In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import cv2
import numpy as np
import sklearn
from glob import glob
from tqdm import tqdm
import pickle, json
import os
from random import shuffle
from matplotlib import pyplot as plt
from scipy.spatial import distance
from sklearn.cluster import DBSCAN
from tqdm import tqdm
from datetime import datetime
import pandas as pd
from itertools import cycle
from time import time

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

from keras.applications.mobilenet import MobileNet
from keras.models import Model, Sequential, load_model as K_load_model
from keras.layers import LSTM, Dense, InputLayer

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

In [9]:
mb_clr = MobileNet(weights=None, input_shape=(224,224,3), classes=2)
mb_bnw = MobileNet(weights=None, input_shape=(224,224,1), classes=2)

In [12]:
mb_bnw.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 1)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 1)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

In [16]:
st = time()
mb_bnw.predict(np.random.rand(1,224,224,1))
print(time() - st)

0.26113176345825195


In [12]:
def get_submodel(main_model, output_layer = "fc1"):
    return Model(inputs=main_model.inputs, output=main_model.get_layer(output_layer).output)
mobile_net_submodel = get_submodel(mobile_net, "global_average_pooling2d_1")

In [33]:
b = np.random.rand(16,112,112,3)
st = time()
mobile_net_new.predict(b)
print(time() - st)

1.261125087738037


In [34]:
def get_submodel(main_model, output_layer = "fc1"):
    return Model(inputs=main_model.inputs, output=main_model.get_layer(output_layer).output)

mobile_net = MobileNet()
mobile_net_submodel = get_submodel(mobile_net, "global_average_pooling2d_1")

version = str(datetime.now())[:16].replace(" ", "_").replace("-", "_").replace(":", "_")

def read_image(image_path):
    return cv2.imread(image_path)

def resize_image(im_array, resolution = (224,224)):
    return cv2.resize(im_array, resolution)

def load_images(files_list):
    return np.array([read_image(file) for file in files_list])

def get_frames_video(video_path, resize_to = (224,224), output_frames = 16):
    cap = cv2.VideoCapture(video_path)
    read_count = 1
    frames_list = list()
    frame_total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    while cap.isOpened():
        isRead, frame = cap.read()
        if not isRead: break
        if read_count % (int(frame_total/output_frames) -1) == 0:
            frame = cv2.resize(frame, resize_to)
            frames_list.append(frame)
        read_count += 1
        if len(frames_list) == 16: break
    return np.array(frames_list)

def get_frame_features(frames_list):
    # return mobile_net_submodel.predict(frames_list)
    return np.array([np.squeeze(mobile_net_submodel.predict(np.expand_dims(frame, axis=0))) for frame in frames_list])

def get_category(idx):
    a = np.zeros(3)
    a[idx] = 1
    return a

def prep_data(data_folder  = "DataSets/KTH/train/", recreate = False):
    if recreate:
        print("Recreating Data . . .")
        for folder in tqdm(os.listdir(data_folder)):
            with open("LSTM_train_data/" + folder+".pkl", "wb") as f:
                pickle.dump({'folder': np.array([get_frame_features(get_frames_video(os.path.join(os.path.join(data_folder, folder), file))) for file in os.listdir(os.path.join(data_folder, folder))])}, f)
            print("Created " + "LSTM_train_data/" + folder+".pkl")
        print("Done creating the necessary  files . . .")
        
        classes = list(set(['walking', 'boxing', 'handwaving']))
        class_id = dict([(cls, get_category(idx)) for idx, cls in enumerate(classes)])

        with open("LSTM_train_data/class.pkl", "wb") as f:
            pickle.dump({"classes": classes, "class_id": class_id}, f)

    trainX, trainY = list(), list()
    with open("LSTM_train_data/class.pkl", "rb") as f:
        class_info = pickle.load(f)
    classes = class_info["classes"]
    class_id = class_info["class_id"]
    
    for cls, label in class_id.items():
        with open("LSTM_train_data/"+cls+".pkl", "rb") as f:
            data_list = pickle.load(f)['folder']
        for dl in data_list:
            if dl.shape == (16,1024):
                trainX.append(dl)
                trainY.append(label)
    trainX, trainY = np.array(trainX), np.array(trainY)
    return classes, class_id, trainX, trainY

def get_lstm_model(no_classes = 3, lstm_hidden_units = 200, image_features_size = 1024, step_size = 16, print_summary = False):
    lstm_model = Sequential()
    lstm_model.add(InputLayer(input_shape = (step_size, image_features_size)))
    lstm_model.add(LSTM(lstm_hidden_units))
    lstm_model.add(Dense(no_classes))
    if print_summary: print(lstm_model.summary())
    return lstm_model

def train_model(lstm_model, trainX, trainY, epochs = 20, batch_size = 2, validation_split = 0.1):
    lstm_model.compile(loss='mean_squared_error', optimizer='adam') #other losses
    lstm_model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, validation_split=validation_split, shuffle=True, verbose=2)
    os.mkdir("trained_models/" + version)
    lstm_model.save("trained_models/" + version + "/" + version + ".hdf5")
    return lstm_model
    
def load_model(model_path = "model_run_1.h5"):
    lstm_model = get_lstm_model()
    lstm_model.load_weights(model_path)
    return lstm_model
                                      
def model_predict(lstm_model, file_path, classes, get_class = False):
    if get_class:
        features = np.expand_dims(get_frame_features(get_frames_video(file_path)), axis=0)
        st = time()
        lstm_output = lstm_model.predict(features)
        print(time() - st)
        a =  classes[np.argmax(lstm_output)]
        return a
    else: return np.squeeze(lstm_model.predict(np.expand_dims(get_frame_features(get_frames_video(file_path)), axis=0))    )
    
def testfile_class(file_name):
    return os.path.split(file_name)[-1].split("_")[1]

def train(no_classes = 3, lstm_hidden_units = 200, recreate_data = False):
    lstm_model = get_lstm_model(no_classes = no_classes, lstm_hidden_units = lstm_hidden_units)
    classes, class_id, trainX, trainY = prep_data(data_folder  = "DataSets/KTH/train/", recreate = recreate_data)
    lstm_model = train_model(lstm_model, trainX, trainY, epochs = 40)
    return lstm_model, classes, class_id

def run_training():
    lstm_model = train(lstm_hidden_units=200, recreate_data = False)

def calc_precision_recall(n_classes, Y_test, y_score, version, plot = True):
    # For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                            y_score[:, i])
        average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])
    
    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(), y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_test, y_score, average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'.format(average_precision["micro"]))
    
    if plot:
        plt.figure()
        plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,
                 where='post')
        plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b')

        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.ylim([0.0, 1.05])
        plt.xlim([0.0, 1.0])
        plt.title('Average precision score, micro-averaged over all classes: AP={0:0.2f}'.format(average_precision["micro"]))
        plt.savefig("trained_models/" + version + "/" + version +"_PR_curve.jpg")
        
    return average_precision["micro"], precision, recall, average_precision

def test_model(version = "2019_11_23_22_18", test_classes_list = ['jogging', 'handclapping', 'running'], test_classes_dict = {'handclapping':1, 'jogging':2, 'running':2}, test_path = "DataSets/KTH/test/*/*", custom_name =""):
    lstm_trained_model = load_model("trained_models/" + version + "/" + version +".hdf5")
#     print(lstm_trained_model.summary())
    lstm_model_sub = get_submodel(lstm_trained_model, 'lstm_1')
    
    test_results = dict()
    for idx, i in enumerate(glob(test_path)):
        try:
            pred = model_predict(lstm_trained_model, i, test_classes_list, get_class = True)
            features =  model_predict(lstm_model_sub, i, test_classes_list)
            act = testfile_class(i)
            test_results[idx] = {"file" : i, "actual" : act, "prediction" : pred, "features": features, "pred_id": test_classes_dict[pred], "act_id": test_classes_dict[act]}
        except Exception as e:
            print(e)
            continue
            
    with open("trained_models/" + version + "/" + version + custom_name + ".pkl", "wb") as f:
        pickle.dump(test_results, f)

    results_df = pd.DataFrame.from_dict(test_results).transpose()
    
    Y_test = np.array([get_category(i) for i in np.array(results_df['act_id'])])
    y_score = np.array([get_category(i) for i in np.array(results_df['pred_id'])])
    
    calc_precision_recall(3, Y_test, y_score, version, True)

ValueError: No such layer: global_average_pooling2d_1

In [ ]:
test_model()

0.25221705436706543
5.295051574707031
0.008090496063232422
4.464651584625244


In [4]:
CVIR_test_path = "DataSets/Final_Data_Repo/*"

version = "2019_11_23_22_18"
lstm_trained_model = load_model("trained_models/" + version + "/" + version +".hdf5")
#     print(lstm_trained_model.summary())
lstm_model_sub = get_submodel(lstm_trained_model, 'lstm_1')

# CVIR_test_videos_features = dict()
# for fl in tqdm(glob(CVIR_test_path)):
#     try:
#         CVIR_test_videos_features[fl] = model_predict(lstm_model_sub, fl, [], get_class = False)
#     except Exception as e:
#         print("ERROR", fl, e)
#         continue
        
# print(len(CVIR_test_videos_features))
# with open("repo.pkl", "wb") as f:
#     pickle.dump(CVIR_test_videos_features, f)

with open("repo.pkl", "rb") as f:
    CVIR_test_videos_features = pickle.load(f)

repo_dict = {'Covering' : 0, 'Uncovering' : 1, "Pushing": 2, "Moving" : 3, "Poking" : 4}

repo_data = list()
for fl, feat in CVIR_test_videos_features.items():
    label = -1
    for k in repo_dict.keys():
        if k in fl:
            label = repo_dict[k]
            break
    repo_data.append(np.array([feat, label]))

repo_data = np.array(repo_data)
repo_data_use, repo_data_test = repo_data[:80], repo_data[80:]

def cosine_similarity(a,b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
def find_closest(test_array, threshold = 0.95):
    close_list = list()
    for idx, (feat, label) in enumerate(repo_data_use):
        if cosine_similarity(feat, test_array) > threshold:
            close_list.append([idx, label, cosine_similarity(feat, test_array)])
    return close_list

print(find_closest(repo_data_test[1][0]), "\n\n", repo_data_test[1][1])

In [76]:
CVIR_test_path = "DataSets/KTH/test/*/*"

version = "2019_11_23_22_18"
lstm_trained_model = load_model("trained_models/" + version + "/" + version +".hdf5")
#     print(lstm_trained_model.summary())
lstm_model_sub = get_submodel(lstm_trained_model, 'lstm_3')

CVIR_test_videos_features = dict()
for fl in tqdm(glob(CVIR_test_path)):
    try:
        CVIR_test_videos_features[fl] = model_predict(lstm_model_sub, fl, [], get_class = False)
    except Exception as e:
        print("ERROR", fl, e)
        continue
        
print(len(CVIR_test_videos_features))
with open("test_ony.pkl", "wb") as f:
    pickle.dump(CVIR_test_videos_features, f)
    
repo_dict = {'handclapping' : 0, 'jogging' : 1, "running": 2}

repo_data = list()
for fl, feat in CVIR_test_videos_features.items():
    label = -1
    for k in repo_dict.keys():
        if k in fl:
            label = repo_dict[k]
            break
    repo_data.append(np.array([feat, label]))

repo_data = np.array(repo_data)
repo_data_use, repo_data_test = repo_data[:80], repo_data[80:]

def cosine_similarity(a,b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
def find_closest(test_array, threshold = 0.95):
    close_list = list()
    for idx, (feat, label) in enumerate(repo_data_use):
        if cosine_similarity(feat, test_array) > threshold:
            close_list.append([idx, label, cosine_similarity(feat, test_array)])
    return close_list

print(find_closest(repo_data_test[1][0]), "\n\n", repo_data_test[1][1])

100%|██████████| 299/299 [15:21<00:00,  3.08s/it]

299
[[2, 2, 0.96807516], [19, 2, 0.9588548], [21, 2, 0.9534637], [29, 1, 0.95916694], [32, 1, 0.98214936], [37, 1, 0.9900864], [40, 2, 0.97153825], [41, 2, 0.98444843], [43, 2, 0.9790073], [46, 1, 0.9510906], [47, 2, 0.97655475], [51, 2, 0.96483916], [55, 2, 0.977283], [56, 2, 0.9763586], [62, 1, 0.96223205], [64, 1, 0.9851486], [65, 1, 0.9854425], [67, 2, 0.97984725], [73, 2, 0.97161907], [74, 2, 0.97578615], [78, 1, 0.9793568]] 

 1


In [89]:
def check_accuracy(retrieved, index):
    correct = 0
    for i in retrieved[:,1]:
        if repo_data_test[index][1] in [1,2] and i in [1,2]:
            correct +=1
        else:
            if i == repo_data_test[index][1]:
                correct += 1
    return correct / len(retrieved)

def retrieve(index):
    retrieved = np.array(find_closest(repo_data_test[index][0], 0.85))
    accuracy = check_accuracy(retrieved, index)
    return accuracy

In [90]:
accuracy_dict = {0:[], 1:[], 2:[]}
for i in range(len(repo_data_test)):
    accuracy_dict[repo_data_test[i][1]].append(retrieve(i))
    
for k, v in accuracy_dict.items():
    print("Accuracy for ", k, " is ", sum(v)/len(v))

Accuracy for  0  is  0.8492067480548211
Accuracy for  1  is  0.9217540665877051
Accuracy for  2  is  0.9403554660252235
